In [1]:
# Import libraries and packages
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
# Load encounters file
dept_path = '/labs/kamaleswaranlab/ECMO/new_data/TAB2_Encounter_Departments.parquet.gzip'
dept = pd.read_parquet(dept_path)
dept[['BIRTH_DATE', 'Hosp_Admission', 'Hosp_Discharge', 'Entered_Dept', 'Exited_Dept']] = dept[['BIRTH_DATE', 'Hosp_Admission', 'Hosp_Discharge', 'Entered_Dept', 'Exited_Dept']].apply(pd.to_datetime)
dept = dept[['Pat ID', 'Encounter CSN', 'Name', 'BIRTH_DATE', 'Department', 'Entered_Dept', 'Exited_Dept', 'Hosp_Admission', 'Hosp_Discharge']]
dept.columns = ['patid', 'csn', 'name', 'dob', 'department', 'entered_dept', 'exited_dept', 'hosp_adm', 'hosp_disch']
dept = dept[(dept['department'].str.contains('PEDIATRIC ICU')) & (dept['hosp_adm'] >= '2010-01-01')]
dept['csn'] = dept['csn'].astype(int)
dept.dropna(inplace=True)
dept.drop_duplicates(inplace=True)

### 1. Extract data

1. Extract data from flowsheets and labs.

In [3]:
# Create Dictionary with variable names for flowsheets
fs_vars = {14: 'Weight',
            2061017: 'Code Sheet Weight (kg)',
            8: 'Pulse',
            9: 'Resp',
            6: 'Temp',
            5: 'BP',
            10: 'SpO2', 
            2059505: 'MAP',
            2059508: 'ABP MAP', 
            2060342: 'FiO2 (%)',
            2065112: 'POC PCO2',
            2065113: 'POC PO2',
            2060631: 'PaO2/FiO2 (Calculated)',
            2059565: 'RLE Capillary Refill (sec)',
            2059561: 'LLE Capillary Refill (sec)',
            2059557: 'RUE Capillary Refill (sec)',
            2059553: 'LUE Capillary Refill (sec)',
            2058121: 'PERIPHERAL VASCULAR WDL',
            2053011: 'Activity',
            2055126: 'Coma Scale Total',
            2060485: 'Tidal Volume Set', 
            2060343: 'Oxygen Flow (lpm)', 
            2053013: 'Pupil Right Reaction',
            2053015: 'Pupil Left Reaction'}

# Create Dictionary with variable names for labs
lab_vars = {2255: 'POTASSIUM', 
            417: 'CALCIUM', 
            1285: 'GLUCOSE', 
            1333: 'HCO3',
            3255: 'WBC', 
            315: 'BAND NEUTROPHILS % (MANUAL)', 
            268: 'ARTERIAL BASE EXCESS',
            3182: 'VENOUS BASE EXCESS',
            426: 'CAP BASE EXCESS',
            267: 'ART BASE DEFICIT',
            3181: 'VENOUS BASE DEFICIT',
            425: 'CAP BASE DEFICIT',
            1700: 'LACTIC ACID', 
            2229: 'POC LACTIC ACID',
            16301: 'LACTIC ACID WHOLE BLOOD', 
            272: 'ARTERIAL POC PH', 
            431: 'CAPILLARY POC PH', 
            3187: 'VENOUS POC PH', 
            5386: 'BUN/CREATININE RATIO QUEST', 
            9266: 'BUN/CREATININE RATIO OSF', 
            5290: 'BUN/CREATININE RATIO LABCORP',
            343: 'BILIRUBIN TOTAL', 
            150: 'ALT (SGPT)', 
            281: 'AST (SGOT)', 
            2206: 'PLATELETS',               
            1621: 'INT NORM RATIO', 
            2306: 'PROTIME',
            271: 'ARTERIAL POC PCO2', 
            273: 'ARTERIAL POC PO2', 
            393: 'BUN',
            733: 'CREATININE', 
            965: 'DDIMER UNITS', 
            1183: 'FIBRINOGEN'}

# List of flowsheets
fs_list = list(filter(lambda x: '.parquet' in x, os.listdir('/labs/kamaleswaranlab/ECMO/new_data/new_flowsheets_feb23')))

# Loop through files to extract variables of dataset picu stays from flowsheets
vars_data = []
for filename in fs_list:
    print('Extracting data from {} file...'.format(filename))
    df = pd.read_parquet(os.path.join('/labs/kamaleswaranlab/ECMO/new_data/new_flowsheets_feb23', filename))
    df = df[['Pat ID', 'Encounter CSN', 'Date of Birth', 'Row ID', 'Row Name', 'Recorded Time', 'Value']]
    df[['Encounter CSN', 'Row ID']] = df[['Encounter CSN', 'Row ID']].astype(int)
    vars_data.extend(df[(df['Row ID'].isin(list(fs_vars.keys()))) & (df['Encounter CSN'].isin(dept['csn'].unique().tolist()))].values)

# Extract variables of dataset patients from labs
print('Extracting data from labs file...')
df = pd.read_parquet('/labs/kamaleswaranlab/ECMO/new_data/DR15269_LABsAndPFTs.parquet.gzip')
df = df[['Pat ID', 'Encounter CSN', 'Date of Birth', 'Component ID', 'Component', 'Result Date', 'Result']]
df.columns = ['Pat ID', 'Encounter CSN', 'Date of Birth', 'Row ID', 'Row Name', 'Recorded Time', 'Value']
df[['Encounter CSN', 'Row ID']] = df[['Encounter CSN', 'Row ID']].astype(int)
vars_data.extend(df[(df['Row ID'].isin(list(lab_vars.keys()))) & (df['Encounter CSN'].isin(dept['csn'].unique().tolist()))].values)

# Create dataframe
print('Creating dataframe...')
variables = pd.DataFrame(vars_data)
variables.columns = ['patid', 'csn', 'dob', 'variable_id', 'variable_name', 'recorded_time', 'value']
variables.reset_index(drop=True, inplace=True)
variables[['dob', 'recorded_time']] = variables[['dob', 'recorded_time']].apply(pd.to_datetime)

# Save dataset file
variables.to_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/raw_variables.parquet.gzip', compression='gzip')

Extracting data from Flowsheet_Rows_2010.parquet.gzip file...
Extracting data from Flowsheet_Rows_2011.parquet.gzip file...
Extracting data from Flowsheet_Rows_2012.parquet.gzip file...
Extracting data from Flowsheet_Rows_2013.parquet.gzip file...
Extracting data from Flowsheet_Rows_2014.parquet.gzip file...
Extracting data from Flowsheet_Rows_2015.parquet.gzip file...
Extracting data from Flowsheet_Rows_2016.parquet.gzip file...
Extracting data from Flowsheet_Rows_2017.parquet.gzip file...
Extracting data from Flowsheet_Rows_2018.parquet.gzip file...
Extracting data from Flowsheet_Rows_2019.parquet.gzip file...
Extracting data from Flowsheet_Rows_2020.parquet.gzip file...
Extracting data from Flowsheet_Rows_2021.parquet.gzip file...
Extracting data from Flowsheet_Rows_2022.parquet.gzip file...
Extracting data from labs file...
Creating dataframe...


2. Fix blood pressure.
3. Add mechanical ventilation data.
4. Add vasoactive agents data.

In [5]:
# Load raw variables
variables = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/raw_variables.parquet.gzip')
variables[['dob', 'recorded_time']] = variables[['dob', 'recorded_time']].apply(pd.to_datetime)
variables['csn'] = variables['csn'].astype(int)

# Fix Sys BP
variables.loc[variables['variable_name'] == 'BP', 'value'] = variables.loc[variables['variable_id'] == 5, 'value'].apply(lambda x: float(str(x).split('/')[0]))

# Fix pupillary reaction
variables.loc[(variables['variable_name'].isin(['Pupil Right Reaction', 'Pupil Left Reaction'])) & (variables['value'].isin(['Brisk', 'Sluggish', 'Hippus'])), 'value'] = 'Reactive'
variables.loc[(variables['variable_name'].isin(['Pupil Right Reaction', 'Pupil Left Reaction'])) & (variables['value'].isin(['Non-reactive'])), 'value'] = 'Non-reactive'
variables.loc[(variables['variable_name'].isin(['Pupil Right Reaction', 'Pupil Left Reaction'])) & (variables['value'].isin(['Unable to assess', 'Pinpoint', 'No eye', 'Pharmacologically dilated', 'Keyhole', 'Ovoid', 'Ovid'])), 'value'] = 'Unable to Assess'

# Add mechanical ventilation data
mv = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/mv_data.parquet.gzip')
mv['csn'] = mv['csn'].astype(int)
mv = mv[mv['csn'].isin(variables['csn'].unique().tolist())]
mv[['dob', 'recorded_time']] = mv[['dob', 'recorded_time']].apply(pd.to_datetime)
variables = pd.concat([variables, mv])

# Add resp support data
resp = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/resp_data.parquet.gzip')
resp['csn'] = resp['csn'].astype(int)
resp = resp[resp['csn'].isin(variables['csn'].unique().tolist())]
resp[['dob', 'recorded_time']] = resp[['dob', 'recorded_time']].apply(pd.to_datetime)
variables = pd.concat([variables, resp])

# Load meds
meds = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/filtered_meds.parquet.gzip')
meds = meds[['patid', 'csn', 'dob', 'med_id', 'med', 'mar_action', 'mar_time']]
meds['csn'] = meds['csn'].astype(int)
meds[['dob', 'mar_time']] = meds[['dob', 'mar_time']].apply(pd.to_datetime)
meds = meds[meds['csn'].isin(variables['csn'].unique().tolist())]
meds.reset_index(inplace=True, drop=True)

# Concat vasoactive agents
meds.columns = ['patid', 'csn', 'dob', 'variable_id', 'variable_name', 'mar_action', 'recorded_time']
meds['value'] = 1.0
vasoactive_list = ['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine', 'adrenaline', 'vasopressin']
variables = pd.concat([variables, meds.loc[(meds['variable_name'].str.contains('|'.join(vasoactive_list), case=False, na=False)) & 
                                    ~ (meds['mar_action'].str.contains('downtime', case=False, na=False)), 
                                    ['patid', 'csn', 'dob', 'variable_id', 'variable_name', 'recorded_time', 'value']]])

variables.loc[variables['variable_name'].str.contains('|'.join(vasoactive_list), case=False, na=False), 'variable_name'] = 'vasoactive'

5. Fix names.

In [6]:
# Fix names
variables.loc[variables['variable_name'] == 'POTASSIUM', 'variable_name'] = 'potassium'
variables.loc[variables['variable_name'] == 'CALCIUM', 'variable_name'] = 'calcium'
variables.loc[variables['variable_name'] == 'GLUCOSE', 'variable_name'] = 'glucose'
variables.loc[variables['variable_name'] == 'HCO3', 'variable_name'] = 'bicarbonate'
variables.loc[variables['variable_name'] == 'Weight', 'variable_name'] = 'weight'
variables.loc[variables['variable_name'] == 'Code Sheet Weight (kg)', 'variable_name'] = 'weight'
variables.loc[variables['variable_name'] == 'Pulse', 'variable_name'] = 'pulse'
variables.loc[variables['variable_name'] == 'Resp', 'variable_name'] = 'resp'
variables.loc[variables['variable_name'] == 'Temp', 'variable_name'] = 'temp'
variables.loc[variables['variable_name'] == 'SpO2', 'variable_name'] = 'spo2'
variables.loc[variables['variable_name'] == 'BP', 'variable_name'] = 'sys_bp'
variables.loc[variables['variable_name'] == 'MAP', 'variable_name'] = 'map'
variables.loc[variables['variable_name'] == 'ABP MAP', 'variable_name'] = 'map'
variables.loc[variables['variable_name'] == 'FiO2 (%)', 'variable_name'] = 'fio2'
variables.loc[variables['variable_name'] == 'Coma Scale Total', 'variable_name'] = 'coma_scale'
variables.loc[variables['variable_name'] == 'WBC', 'variable_name'] = 'wbc'
variables.loc[variables['variable_name'] == 'BAND NEUTROPHILS % (MANUAL)', 'variable_name'] = 'band_neutrophils'
variables.loc[variables['variable_name'] == 'ARTERIAL BASE EXCESS', 'variable_name'] = 'base_excess'
variables.loc[variables['variable_name'] == 'VENOUS BASE EXCESS', 'variable_name'] = 'base_excess'
variables.loc[variables['variable_name'] == 'CAP BASE EXCESS', 'variable_name'] = 'base_excess'
variables.loc[variables['variable_name'] == 'ART BASE DEFICIT', 'variable_name'] = 'base_deficit'
variables.loc[variables['variable_name'] == 'VENOUS BASE DEFICIT', 'variable_name'] = 'base_deficit'
variables.loc[variables['variable_name'] == 'CAP BASE DEFICIT', 'variable_name'] = 'base_deficit'
variables.loc[variables['variable_name'] == 'LACTIC ACID', 'variable_name'] = 'lactic_acid'
variables.loc[variables['variable_name'] == 'POC LACTIC ACID', 'variable_name'] = 'lactic_acid'
variables.loc[variables['variable_name'] == 'LACTIC ACID WHOLE BLOOD', 'variable_name'] = 'lactic_acid'
variables.loc[variables['variable_name'] == 'ARTERIAL POC PH', 'variable_name'] = 'art_ph'
variables.loc[variables['variable_name'] == 'CAPILLARY POC PH', 'variable_name'] = 'cap_ph'
variables.loc[variables['variable_name'] == 'VENOUS POC PH', 'variable_name'] = 'venous_ph'
variables.loc[variables['variable_name'] == 'BUN/CREATININE RATIO QUEST', 'variable_name'] = 'bun_creat'
variables.loc[variables['variable_name'] == 'BUN/CREATININE RATIO OSF', 'variable_name'] = 'bun_creat'
variables.loc[variables['variable_name'] == 'BUN/CREATININE RATIO LABCORP', 'variable_name'] = 'bun_creat'
variables.loc[variables['variable_name'] == 'BILIRUBIN TOTAL', 'variable_name'] = 'bilirubin'
variables.loc[variables['variable_name'] == 'ALT (SGPT)', 'variable_name'] = 'alt'
variables.loc[variables['variable_name'] == 'AST (SGOT)', 'variable_name'] = 'ast'
variables.loc[variables['variable_name'] == 'PLATELETS', 'variable_name'] = 'platelets'
variables.loc[variables['variable_name'] == 'INT NORM RATIO', 'variable_name'] = 'inr'
variables.loc[variables['variable_name'] == 'PROTIME', 'variable_name'] = 'pt'
variables.loc[variables['variable_name'] == 'BUN', 'variable_name'] = 'bun'
variables.loc[variables['variable_name'] == 'CREATININE', 'variable_name'] = 'creatinine'
variables.loc[variables['variable_name'] == 'POC PCO2', 'variable_name'] = 'paco2'
variables.loc[variables['variable_name'] == 'ARTERIAL POC PCO2', 'variable_name'] = 'paco2'
variables.loc[variables['variable_name'] == 'POC PO2', 'variable_name'] = 'pao2'
variables.loc[variables['variable_name'] == 'ARTERIAL POC PO2', 'variable_name'] = 'pao2'
variables.loc[variables['variable_name'] == 'PaO2/FiO2 (Calculated)', 'variable_name'] = 'pao2_fio2'
variables.loc[variables['variable_name'] == 'RLE Capillary Refill (sec)', 'variable_name'] = 'cap_refill'
variables.loc[variables['variable_name'] == 'LLE Capillary Refill (sec)', 'variable_name'] = 'cap_refill'
variables.loc[variables['variable_name'] == 'RUE Capillary Refill (sec)', 'variable_name'] = 'cap_refill'
variables.loc[variables['variable_name'] == 'LUE Capillary Refill (sec)', 'variable_name'] = 'cap_refill'
variables.loc[variables['variable_name'] == 'PERIPHERAL VASCULAR WDL', 'variable_name'] = 'periph_vasc'
variables.loc[variables['variable_name'] == 'Activity', 'variable_name'] = 'activity'
variables.loc[variables['variable_name'] == 'Tidal Volume Set', 'variable_name'] = 'tidal_vol'
variables.loc[variables['variable_name'] == 'Oxygen Flow (lpm)', 'variable_name'] = 'o2_flow'
variables.loc[variables['variable_name'] == 'DDIMER UNITS', 'variable_name'] = 'ddimer'
variables.loc[variables['variable_name'] == 'FIBRINOGEN', 'variable_name'] = 'fibrinogen'
variables.loc[variables['variable_name'] == 'Pupil Right Reaction', 'variable_name'] = 'pupil_right_reaction'
variables.loc[variables['variable_name'] == 'Pupil Left Reaction', 'variable_name'] = 'pupil_left_reaction'

variables.to_pickle('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/variables.pkl')